this script seeks to replicate the work of xxx who has explained how to classify video data in his venerable blog post
https://medium.com/smileinnovation/training-neural-network-with-image-sequence-an-example-with-video-as-input-c3407f7a0b0f
1. Activity recognition


hail patrice!

In [12]:
# path where data lives
DATA_PATH = 'human_activity_data/extracted_videos'

<img src="files/cnn_gru.PNG">

In [13]:
import os
import glob
import keras
# BUG overcome: tensforflow requires visual c++ for some dll file that can be found only there

In [14]:
from keras_video import VideoFrameGenerator

In [15]:
# use sub directories names as classes
classes = [i.split(os.path.sep)[1] for i in glob.glob('human_activity_data/extracted_videos/*')]
classes.sort()

In [16]:
classes

['dribble', 'golf', 'kick_ball']

In [17]:
# some global params
SIZE = (112, 112) # height and width of frame pxl by pxl
CHANNELS = 3 # RGB or whatever
NBFRAME = 5 # num frames in sequence 
BS = 8 # Batch size

In [23]:
# pattern to get videos and classes
glob_pattern='human_activity_data/extracted_videos/{classname}/*.avi'

In [19]:
# for data augmentation
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip=True,
    rotation_range=8,
    width_shift_range=.2,
    height_shift_range=.2)

In [28]:
# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.33, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=data_aug,
    use_frame_cache=True)

class dribble, validation count: 47, train count: 98
class golf, validation count: 34, train count: 71
class kick_ball, validation count: 42, train count: 86
Total data: 3 classes for 255 files for train


In [29]:
# getting validation data
valid = train.get_validation_generator()

Total data: 3 classes for 123 files for validation


In [31]:
# import keras_video.utils
# keras_video.utils.show_sample(train)

Steps:
1. A ConvNet is created and Time distributed to detect “features”
2. The Time distributed output is injected to GRU or LSTM to treat “time series”
3. A DenseNet is then applied to take the decision, to “classify”

## BUILD CONV NET

In [32]:
from keras.layers import Conv2D, BatchNormalization, \
    MaxPool2D, GlobalMaxPool2D
def build_convnet(shape=(112, 112, 3)):
    # specify momentum
    momentum = .9
    # cerate the model from 9=(what is sequential: not clear, just a basic layer stacker class)
    model = keras.Sequential()
    # builds a 2D convnet, filter #, filter size ...
    model.add(Conv2D(64, (3,3), input_shape=shape,
        padding='same', activation='relu'))
    
    # same takes in previous layer, I guess
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    # adding batch normalization (forgot what batch normalization is used for)
    model.add(BatchNormalization(momentum=momentum))
    
    # max pool to this layer
    model.add(MaxPool2D())
    
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    # flatten... (not sure why not flatten instead of GlobalMaxPool2D)
    model.add(GlobalMaxPool2D())
    return model

## Build GRU

In [34]:
from keras.layers import TimeDistributed, GRU, Dense, Dropout
# Shape (5, 112, 112, 3) 5 - time sequence length 112x112 = height vs width 3 - num channels
def action_model(shape=(5, 112, 112, 3), nbout=3):
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_convnet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (5, 112, 112, 3) shape
    # KEY = allows you to add a time sequence to a layer one at a time
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))
    return model

## this is where you tell the model how to train - loss function, weight update mechanism


In [35]:
INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (5, 112, 112, 3)
# what is action model?
model = action_model(INSHAPE, len(classes))

# this is where you tell the model how to train - loss function, weight update mechanism
optimizer = keras.optimizers.Adam(0.001)
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)

## epochs, call backs

In [36]:
EPOCHS=50
# create a "chkp" directory before to run that
# because ModelCheckpoint will write models inside
callbacks = [
    keras.callbacks.ReduceLROnPlateau(verbose=1),
    keras.callbacks.ModelCheckpoint(
        'chkp/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        verbose=1),
]
model.fit_generator(
    train,
    validation_data=valid,
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks
)

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
31/31 [==============================] - 143s 5s/step - loss: 1.1795 - acc: 0.3550 - val_loss: 1.0161 - val_acc: 0.4000

Epoch 00001: saving model to chkp\weights.01-1.02.hdf5
Epoch 2/50
31/31 [==============================] - 133s 4s/step - loss: 1.1560 - acc: 0.3511 - val_loss: 0.9906 - val_acc: 0.5333

Epoch 00002: saving model to chkp\weights.02-0.99.hdf5
Epoch 3/50
31/31 [==============================] - 133s 4s/step - loss: 1.1365 - acc: 0.4197 - val_loss: 0.9303 - val_acc: 0.5917

Epoch 00003: saving model to chkp\weights.03-0.93.hdf5
Epoch 4/50
31/31 [==============================] - 133s 4s/step - loss: 1.0204 - acc: 0.5169 - val_loss: 0.9534 - val_acc: 0.6083

Epoch 00004: saving model to chkp\weights.04-0.95.hdf5
Epoch 5/50
31/31 [==============================] - 134s 4s/step - loss: 0.9230 - acc: 0.5848 - val_loss: 1.1104 - val_acc: 0.3917

Epoch 00005: saving model to chkp\weights.05-1.11.hdf5
Epoch 6/50
31/31 [==============================] - 134s 4s/step 